In [1]:
#flattens the nested directory into a single level 
def flatten_dict(dd, separator='_', prefix=''):
    return {f"{prefix}{separator}{k}" if prefix else k: v
            for kk, vv in dd.items()
            for k, v in (flatten_dict(vv, separator, kk).items() if isinstance(vv, dict) else {kk: vv}.items())}

In [2]:
import requests

def call_api(year,month,day):
    

    url = "https://partners.api.skyscanner.net/apiservices/v3/flights/indicative/search"

    payload = { "query": {
            "market": "UK",
            "locale": "en-GB",
            "currency": "EUR",
            "queryLegs": [
                {
                    "originPlace": { "queryPlace": { "iata": orig } },
                    "destinationPlace": { "queryPlace": { "iata": dest } },
                    "fixedDate": {
                        "year": year,
                        "month": month,
                        "day": day
                    }
                }
            ]
        } }
    headers = {
        "x-api-key": "prtl6749387986743898559646983194"
    }

    response = requests.post(url, json=payload, headers=headers)
    output=response.json()
    results=output["content"]["results"]
    import pandas as pd
    import json

    quotes=results["quotes"]
    quotes_df=pd.json_normalize(quotes)
    quotes_df.columns=[col.split(".")[-1] for col in quotes_df.columns]

    flattened_results=[flatten_dict(result) for result in quotes.values()]
    
    quote_df = pd.DataFrame.from_records(flattened_results)
    return quote_df

In [3]:
#USER'S CHOICE OF DATES
orig = input("Origin IATA: ")
dest = input("Destiny IATA: ")
date_from=input("Search date from (dd/mm/yyyy): ")
date_to=input("Search date to (dd/mm/yyyy): ")

Origin IATA:  LIS
Destiny IATA:  POZ
Search date from (dd/mm/yyyy):  22/06/2023
Search date to (dd/mm/yyyy):  30/06/2023


In [4]:
#FATES AFTER USER CHOICE 
from datetime import datetime
import pandas as pd

date_from = pd.to_datetime(date_from, format='%d/%m/%Y')
date_to = pd.to_datetime(date_to, format='%d/%m/%Y')

dates = pd.date_range(date_from, date_to, freq='D')

In [5]:
dates

DatetimeIndex(['2023-06-22', '2023-06-23', '2023-06-24', '2023-06-25',
               '2023-06-26', '2023-06-27', '2023-06-28', '2023-06-29',
               '2023-06-30'],
              dtype='datetime64[ns]', freq='D')

In [6]:
dates[0].year

2023

In [7]:
dates[0].month

6

In [8]:
len(dates)

9

In [9]:
quote_df_list=[]
    
for i in range(0,len(dates)):
    orig=orig
    dest=dest
    year=dates[i].year
    month=dates[i].month
    day=dates[i].day
    

    quote_df_list.append(call_api(year,month,day))

In [10]:
quote_df_list[1]

,minPrice_amount,minPrice_unit,minPrice_updateStatus,isDirect,outboundLeg_originPlaceId,outboundLeg_destinationPlaceId,outboundLeg_departureDateTime_year,outboundLeg_departureDateTime_month,outboundLeg_departureDateTime_day,outboundLeg_departureDateTime_hour,...,inboundLeg_originPlaceId,inboundLeg_destinationPlaceId,inboundLeg_departureDateTime_year,inboundLeg_departureDateTime_month,inboundLeg_departureDateTime_day,inboundLeg_departureDateTime_hour,inboundLeg_departureDateTime_minute,inboundLeg_departureDateTime_second,inboundLeg_quoteCreationTimestamp,inboundLeg_marketingCarrierId
0,48,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,False,95565055,128667756,2023,6,23,0,...,,,0,0,0,0,0,0,0,


In [11]:
for i in range (0,len(quote_df_list)):
    if len(quote_df_list[i]) > 1:
        quote_df_list[i]['minPrice_amount'] = pd.to_numeric(quote_df_list[i]['minPrice_amount'])
        min_idx = quote_df_list[i]['minPrice_amount'].idxmin()
        quote_df_list[i]=quote_df_list[i].loc[[min_idx]]

In [12]:
quote_df_list[1]

,minPrice_amount,minPrice_unit,minPrice_updateStatus,isDirect,outboundLeg_originPlaceId,outboundLeg_destinationPlaceId,outboundLeg_departureDateTime_year,outboundLeg_departureDateTime_month,outboundLeg_departureDateTime_day,outboundLeg_departureDateTime_hour,...,inboundLeg_originPlaceId,inboundLeg_destinationPlaceId,inboundLeg_departureDateTime_year,inboundLeg_departureDateTime_month,inboundLeg_departureDateTime_day,inboundLeg_departureDateTime_hour,inboundLeg_departureDateTime_minute,inboundLeg_departureDateTime_second,inboundLeg_quoteCreationTimestamp,inboundLeg_marketingCarrierId
0,48,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,False,95565055,128667756,2023,6,23,0,...,,,0,0,0,0,0,0,0,


In [13]:
df = pd.concat(quote_df_list, ignore_index=True)
df

,minPrice_amount,minPrice_unit,minPrice_updateStatus,isDirect,outboundLeg_originPlaceId,outboundLeg_destinationPlaceId,outboundLeg_departureDateTime_year,outboundLeg_departureDateTime_month,outboundLeg_departureDateTime_day,outboundLeg_departureDateTime_hour,...,inboundLeg_originPlaceId,inboundLeg_destinationPlaceId,inboundLeg_departureDateTime_year,inboundLeg_departureDateTime_month,inboundLeg_departureDateTime_day,inboundLeg_departureDateTime_hour,inboundLeg_departureDateTime_minute,inboundLeg_departureDateTime_second,inboundLeg_quoteCreationTimestamp,inboundLeg_marketingCarrierId
0,63,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,False,95565055,128667756,2023,6,22,0,...,,,0,0,0,0,0,0,0,
1,48,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,False,95565055,128667756,2023,6,23,0,...,,,0,0,0,0,0,0,0,
2,150,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,False,95565055,128667756,2023,6,25,0,...,,,0,0,0,0,0,0,0,
3,68,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,False,95565055,128667756,2023,6,26,0,...,,,0,0,0,0,0,0,0,
4,60,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,False,95565055,128667756,2023,6,27,0,...,,,0,0,0,0,0,0,0,
5,46,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,False,95565055,128667756,2023,6,28,0,...,,,0,0,0,0,0,0,0,
6,52,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,False,95565055,128667756,2023,6,29,0,...,,,0,0,0,0,0,0,0,
7,61,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,False,95565055,128667756,2023,6,30,0,...,,,0,0,0,0,0,0,0,


In [14]:
dates_df = pd.DataFrame({'Date': dates})

In [15]:
# result for the range of the dates
result = pd.concat([dates_df, df], axis=1)
result

,Date,minPrice_amount,minPrice_unit,minPrice_updateStatus,isDirect,outboundLeg_originPlaceId,outboundLeg_destinationPlaceId,outboundLeg_departureDateTime_year,outboundLeg_departureDateTime_month,outboundLeg_departureDateTime_day,...,inboundLeg_originPlaceId,inboundLeg_destinationPlaceId,inboundLeg_departureDateTime_year,inboundLeg_departureDateTime_month,inboundLeg_departureDateTime_day,inboundLeg_departureDateTime_hour,inboundLeg_departureDateTime_minute,inboundLeg_departureDateTime_second,inboundLeg_quoteCreationTimestamp,inboundLeg_marketingCarrierId
0,2023-06-22,63,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,False,95565055,128667756,2023.0,6.0,22.0,...,,,0.0,0.0,0.0,0.0,0.0,0.0,0,
1,2023-06-23,48,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,False,95565055,128667756,2023.0,6.0,23.0,...,,,0.0,0.0,0.0,0.0,0.0,0.0,0,
2,2023-06-24,150,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,False,95565055,128667756,2023.0,6.0,25.0,...,,,0.0,0.0,0.0,0.0,0.0,0.0,0,
3,2023-06-25,68,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,False,95565055,128667756,2023.0,6.0,26.0,...,,,0.0,0.0,0.0,0.0,0.0,0.0,0,
4,2023-06-26,60,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,False,95565055,128667756,2023.0,6.0,27.0,...,,,0.0,0.0,0.0,0.0,0.0,0.0,0,
5,2023-06-27,46,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,False,95565055,128667756,2023.0,6.0,28.0,...,,,0.0,0.0,0.0,0.0,0.0,0.0,0,
6,2023-06-28,52,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,False,95565055,128667756,2023.0,6.0,29.0,...,,,0.0,0.0,0.0,0.0,0.0,0.0,0,
7,2023-06-29,61,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,False,95565055,128667756,2023.0,6.0,30.0,...,,,0.0,0.0,0.0,0.0,0.0,0.0,0,
8,2023-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
